<a href="https://colab.research.google.com/github/sergo2/gigachat_tutorials/blob/main/%D0%97%D0%B0%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузчики документов для RAG и других цепочек GigaChain/LangChain

Создание вопросно-ответных систем или иных приложений, где большая языковая модель взаимодействует с контекстом, нужны подготовленные тексты из внешних источников. Очень часто источниками этих текстов являются файлы различных форматов. В этом туториале мы познакомимся с работой некоторых загрузчиков, которые есть в библиотеке GigaChain/LangChain, позволяющих извлекать тексты из файлов. Мы рассмотрим примеры как обычных текстовых файлов, так и табличных документов. Загрузчики представляют собой интеграции внешних библиотек. В результате при работе с любыми источниками используется один и тот же интерфейс и единый формат выдаваемых на выходе данных, что облегчает разработку приложений.

На основе извлеченных текстовых данных мы создадим векторные базы данных с эмбеддингами, а затем сделаем ряд запросов к этим базам данных, чтобы проверить, насколько возвращаемые фрагменты релевантны запросам.

Давайте приступим!

## Установим необходимые библиотеки

In [14]:
!pip install gigachain gigachain-community unstructured[all-docs] faiss-cpu sentence-transformers --quiet

## Текстовые данные
В этом разделе мы рассмотрим загрузчики для текстовых данных: PDF и DOCX файлов.

В арсенале GigaChain есть несколько загрузчиков под каждый из этих типов файлов. Мы воспользуемся `UnstructuredWordDocumentLoader` и `UnstructuredPDFLoader` для DOCX и PDF соответственно. Другие варианты загрузчиков можно посмотреть по ссылкам:
- [DOCX](https://python.langchain.com/v0.1/docs/integrations/document_loaders/microsoft_word/);
- [PDF](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/).

В качестве текстов используем статью:

*Голиков А.А., Акимов Д.А., Романовский М.С., Тращенков С.В. — Аспекты создания корпоративной вопросно-ответной системы с использованием генеративных предобученных языковых моделей // Litera. – 2023. – No 12. – С. 190 - 205. DOI: 10.25136/2409-8698.2023.12.69353 EDN: FSTHRW URL: https://nbpublish.com/library_read_article.php?id=69353*


### Создание фрагментов на основе PDF

Для подготовки фрагментов используем `RecursiveCharacterTextSplitter`. Зададим приблизительный размер фрагмента и количество символов перекрытия. Сплиттер можно передать в метод `load_and_split()`. На выходе получим список документов `Document`.

In [32]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=700,
                                              chunk_overlap=100)
loader = PyPDFLoader("https://api.hostize.com/files/IWrHyUJA_M/download/file.pdf")

splitted_data = loader.load_and_split(text_splitter)

Посмотрим сколько фрагментов получилось.

In [33]:
len(splitted_data)

41

Выведем на экран два соседних фрагмента, чтобы показать перекрытие.

In [38]:
splitted_data[0]

Document(metadata={'source': 'https://api.hostize.com/files/IWrHyUJA_M/download/file.pdf', 'page': 0}, page_content='1 \n \nЦЕННЫЕ БУМАГИ, СОСТАВЛЯЮЩИЕ НАСТОЯЩИЙ ВЫПУСК, ЯВЛЯЮТСЯ ЦЕННЫМИ \nБУМАГАМИ, ПРЕДНАЗНАЧЕННЫМИ ДЛЯ КВАЛИФИЦИРОВАННЫХ ИНВЕСТОРОВ, И \nОГРАНИЧЕНЫ В ОБОРОТЕ В СООТВЕТСТВИИ С ЗАКОНОДАТЕЛЬСТВОМ РОССИЙСКОЙ \nФЕДЕРАЦИИ \n \nСООБЩЕНИЕ О КЛЮЧЕВЫХ УСЛОВИЯХ ВЫПУСКА \nСТРУКТУРНЫХ ОБЛИГАЦИЙ АО «СБЕРБАНК КИБ» \n \nДата приказа  Президента АО «Сбербанк КИБ»  об утверждении настоящего Сообщения  о ключевых условиях \nвыпуска структурных облигаций АО «Сбербанк КИБ» (далее – «Сообщение»): 29 октября 2024 г. \n \nВид, категория (тип), серия, номер и дата государственной регистрации выпуска ценных бумаг и иные \nидентификационные признаки размещаемых ценных бумаг: структурные процентные дисконтные')

In [28]:
splitted_data[2]

Document(metadata={'source': 'https://file.com.ru/download/zeyT7UoerDfBCoC/qLQ3668KK319j/SKUV.pdf', 'page': 0}, page_content='Интернет исключительно для информирования потенциальных покупателей – квалифицированных инвесторов \nОблигаций о сведениях о ключевых условиях выпуска структурных облигаций Эмитента (далее – «СО», \n«Облигации») и не может быть воспроизведен, передан или распространен далее представителям СМИ или любому \nиному лицу либо опубликован полностью или частично для каких-либо целей. \nСодержание настоящего Сообщения о ключевых условиях выпуска не является рекламой СО Эмитента или \nпредложением, обязательств ом, рекомендацией, индивидуальной инвестиционной рекомендацией, побуждением \nсовершать операции на финансовом рынке. Несмотря на получение информации из Сообщения о ключевых \nусловиях выпуска, Вы самостоятельно принимаете все инвестиционные решения и обеспеч иваете соответствие \nтаких решений Вашему инвестиционному профилю в целом и в частности Вашим личным пре

### Создание векторных представлений и векторного хранилища

В этом туториале опять используем модель [`paraphrase-multilingual-mpnet-base-v2`](https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2) для векторизации фрагментов.

В качестве хранилища опять используем `FAISS`. Помимо этой векторной БД в GigaChain/LangChain есть большое количество интеграций с векторными хранилищами, как локальными так и облачными.
Перечень доступных векторных хранилищ можно посмотреть [здесь](https://python.langchain.com/v0.2/docs/integrations/vectorstores/).

In [19]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS

In [35]:
%%time
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs,
                                  encode_kwargs=encode_kwargs)

vector_store = FAISS.from_documents(splitted_data, embedding=embedding)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


CPU times: user 18.4 s, sys: 1.82 s, total: 20.2 s
Wall time: 21.4 s


### Поиск по векторной БД релевантных фрагментов для запросов пользователя
Зададим несколько вопросов по тексту статьи.

- Какой размер чанков показал наибольшую точность ответов при использовании фреймворка Haystack?
- Какие метрики использовались для оценки эффективности использования генеративных предобученных языковых моделей в статье?
- Имя какого английского писателя упомянается в статье?

In [36]:
q1 = 'Кто эмитент структурной облигации CIB-CO-EQ-001S-09?'
q2 = 'Какой базовый актив в структурной облигации CIB-CO-EQ-001S-09?'
q3 = 'Какие даты выплаты структурного дохода?'

#### Вопрос 1
Какой размер чанков показал наибольшую точность ответов при использовании фреймворка Haystack?

#### Правильный ответ
Размер чанков 100, 200 и 1000 токенов показал одинаковую наибольшую точность ответов (0.7) при использовании фреймворка Haystack.

In [22]:
vector_store.similarity_search(q1)

[Document(metadata={'source': 'https://file.com.ru/download/zeyT7UoerDfBCoC/qLQ3668KK319j/SKUV.pdf', 'page': 1}, page_content='2 \n \nСообщение о ключевых условиях выпуска является документом, предназначенным для инфо рмирования \nпотенциальных покупателей Облигаций  – квалифицированных инвесторов об условиях выпуска СО и решениях, \nпринятых Эмитентом в отношении выпуска СО. \nВозврат инвесторам номинальной стоимости СО и/или выплата структурного дохода по СО зависит от наступления \nили не наступления обстоятельств, указанных в эмиссионной документации СО, такие обстоятельства в частности \nмогут включать в себя наступление определенных событий (далее – «Барьерное событие») в связи с \nнеблагоприятным изменением цены (значений) Базовых активов. Таким образом риск инвестора проявляется в \nнеблагоприятном изменении цены ( значений) Базовых активов - в том числе из -за неблагоприятного изменения \nполитической ситуации, резкой девальвации национальной валюты, кризиса рынка государствен

#### Вопрос 2
Какие метрики использовались для оценки эффективности использования генеративных предобученных языковых моделей в статье?

#### Правильный ответ
Для оценки эффективности использования генеративных предобученных языковых моделей использовалась метрика EM (exact match).

In [23]:
vector_store.similarity_search(q2)

[Document(metadata={'source': 'https://file.com.ru/download/zeyT7UoerDfBCoC/qLQ3668KK319j/SKUV.pdf', 'page': 2}, page_content='на «Биржу Базового актива» будут считаться ссылками на Новую Биржу \nБазового актива.  \nВ случае дробления, консолидации или конвертации Базового актива, цены \nБазового актива (включая пересмотр Начальной цены Базового актива) \nопределяются с учётом коэффициента дробления, консолидации или \nконвертации, указанного в общедоступной информации. \nВ случае слияния, поглощения или иного события, в результате которого \nБазовый актив («Заменяемый Базовый актив») перестаёт существовать (кроме \nбанкротства эмитента Базового актива), делистинга Базового актива, а также в \nслучае наступления обстоятельств, результатом которых стало отсутствие \nторгов Базовым активом на Бирже Базового актива в период 8 \nпоследовательных Рабочих дней, в которые торги Базовым активом должны бы \nбыли осуществляться в отсутствие указанных в настоящем абзаце \nобстоятельств, Базовым а

#### Вопрос 3
Имя какого английского писателя упоминается в статье?

#### Правильный ответ
В статье упоминается английский писатель Джордж Гордон Байрон (лорд Байрон).

In [39]:
vector_store.similarity_search(q3)

[Document(metadata={'source': 'https://api.hostize.com/files/IWrHyUJA_M/download/file.pdf', 'page': 4}, page_content='4.2.  Даты выплаты \nструктурного дохода \nПорядковый номер \nДаты выплаты \nструктурного дохода, n Дата выплаты структурного доходаn \n1 4 марта 2025 г. \n2 2 июня 2025 г. \n3 1 сентября 2025 г. \n4 2 декабря 2025 г. \n \n4.3.  Размер структурного \nдохода \n6.250% \n4.4.  Порядок определения \nДат оценки в случае \nприменения \nДополнительных \nпроцедур определения \nцены Базового актива  \nВ случае применения Дополнительных процедур определения цены Базового \nактива, соответствующей Датой оценки является дата, в которую цена Базового \nактива была определена в порядке, изложенном в разделе «Дополнительные \nпроцедуры определения цены Базового актива. \n4.5.  Случаи и порядок, \nкогда Расчетный агент'),
 Document(metadata={'source': 'https://api.hostize.com/files/IWrHyUJA_M/download/file.pdf', 'page': 4}, page_content='ненаступлении иных событий (обстоятельств) не по

### Создание фрагментов на основе DOCX
Используем эту же статью, но в формате DOCX. В качестве загрузчика у нас будет `UnstructuredWordDocumentLoader`.

In [25]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
loader = UnstructuredWordDocumentLoader('aspekty-sozdaniya-korporativnoy-voprosno-otvetnoy-sistemy-s-ispolzovaniem-generativnyh-predobuchennyh-yazykovyh-modeley.docx')
splitted_docx = loader.load_and_split(text_splitter)

FileNotFoundError: no such file aspekty-sozdaniya-korporativnoy-voprosno-otvetnoy-sistemy-s-ispolzovaniem-generativnyh-predobuchennyh-yazykovyh-modeley.docx

Проверим, сколько вышло фрагментов.

In [ ]:
len(splitted_docx)

Видим, что количество фрагментов не совпадает. Сравним содержимое фрагментов из pdf и docx.

In [ ]:
splitted_data[10]

In [ ]:
splitted_docx[10]

In [ ]:
splitted_docx[12]

In [ ]:
docs = embedding.embed_documents([splitted_data[10].page_content, splitted_docx[12].page_content])

In [ ]:
docs

In [ ]:
import numpy as np
cosine_similarity = np.dot(docs[0], docs[1]) / (np.linalg.norm(docs[0]) * np.linalg.norm(docs[1]))

print(f"Косинусное сходство: {cosine_similarity}")

## Создание векторных представлений и векторного хранилища

Создадим векторное хранилище, но уже из фрагментов docx-файла. Затем зададим такие же вопросы.

In [ ]:
vector_store_docx = FAISS.from_documents(splitted_docx, embedding=embedding)

#### Вопрос 1
Какой размер чанков показал наибольшую точность ответов при использовании фреймворка Haystack?

#### Правильный ответ
Размер чанков 100, 200 и 1000 токенов показал одинаковую наибольшую точность ответов (0.7) при использовании фреймворка Haystack.

In [ ]:
vector_store_docx.similarity_search(q1)

#### Вопрос 2
Какие метрики использовались для оценки эффективности использования генеративных предобученных языковых моделей в статье?

#### Правильный ответ
Для оценки эффективности использования генеративных предобученных языковых моделей использовалась метрика EM (exact match).

In [ ]:
vector_store_docx.similarity_search(q2)

#### Вопрос 3
Имя какого английского писателя упоминается в статье?

#### Правильный ответ
В статье упоминается английский писатель Джордж Гордон Байрон (лорд Байрон).

In [ ]:
vector_store_docx.similarity_search(q3)

## Табличные данные
Теперь рассмотрим стандартные форматы табличных файлов: CSV и XLSX.

Для этого используем загрузчики `CSVLoader` и `UnstructuredExcelLoader`.

В качестве табличного файла используем документ ["Динамика уровня и структуры затрат организаций на рабочую силу"](https://rosstat.gov.ru/labour_costs#) с сайта Росстата.

Файл представляет собой несколько таблиц в основном с количественными данными. Поскольку файл изначально в формате XLSX, сделаем из него и документы в формате CSV. Так как в XLSX-файле несколько листов, придется создать отдельный CSV-файл под каждый лист.

In [ ]:
import pandas as pd

# Загрузите Excel файл
file_path = '/content/din-zatrat-rs.xlsx'

# Читаем все листы из Excel файла
sheets = pd.read_excel(file_path, sheet_name=None)

# Сохраняем каждый лист как отдельный CSV файл
for sheet_name, df in sheets.items():
    csv_file = f"{sheet_name}.csv"
    df.to_csv(csv_file, index=False)
    print(f"Сохранено: {csv_file}")


### Создание списка csv-файлов и загрузка
Поскольку загрузчик `CSVLoader` принимает документы по одному, делаем загрузку в цикле.

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from glob import glob
files = glob('*.csv')

splitted_csv = []
for f in files:
  loader = CSVLoader(f)
  splitted_csv.extend(loader.load())

Проверим, сколько фрагментов получилось.

In [ ]:
len(splitted_csv)

Посмотрим отдельный фрагмент.

In [ ]:
splitted_csv[139]

### Создание векторных представлений и векторного хранилища
Поместим фрагменты в базу данных.

In [ ]:
vector_store_csv = FAISS.from_documents(splitted_csv, embedding=embedding)

### Поиск по векторной БД релевантных фрагментов для запросов пользователя
Зададим несколько вопросов по данным таблиц.
- Какой процент расходов на социальную защиту работников в добыче полезных ископаемых был зафиксирован в 2021 году?
- Какой процент от общих затрат на рабочую силу составляла заработная плата в 2005 году по всем обследованным видам экономической деятельности?
- В каком году был зафиксирован наибольший процент заработной платы в обрабатывающей промышленности и чему он равен?

In [ ]:
q4 = 'Какой процент расходов на социальную защиту работников в добыче полезных ископаемых был зафиксирован в 2021 году?'
q5 = 'Какой процент от общих затрат на рабочую силу составляла заработная плата в 2005 году по всем обследованным видам экономической деятельности?'
q6 = 'В каком году был зафиксирован наибольший процент заработной платы в обрабатывающей промышленности и чему он равен?'

#### Вопрос 4
Какой процент расходов на социальную защиту работников в добыче полезных ископаемых был зафиксирован в 2021 году?

#### Правильный ответ
В 2021 году расходы на социальную защиту работников в добыче полезных ископаемых составляли 22,6%.

In [ ]:
vector_store_csv.similarity_search(q4)

#### Вопрос 5
Какой процент от общих затрат на рабочую силу составляла заработная плата в 2005 году по всем обследованным видам экономической деятельности?

#### Правильный ответ
В 2005 году заработная плата составляла 76,6% от общих затрат на рабочую силу по всем обследованным видам экономической деятельности.

In [ ]:
vector_store_csv.similarity_search(q5)

#### Вопрос 6
В каком году был зафиксирован наибольший процент заработной платы в обрабатывающей промышленности и чему он равен?

#### Правильный ответ
Наибольший процент заработной платы в обрабатывающей промышленности был зафиксирован в 2007 году и составил 79,5%.

In [ ]:
vector_store_csv.similarity_search(q6)

### Загрузка табличных данных из xlsx-файла
Для сравнения мы выполним разделение на фрагменты тремя способами:
- используем сплиттер из текстовых примеров (1000 символов на фрагмент и 100 символов на перекрытие);
- применим сплиттер по умолчанию;
- зададим параметр `mode='elements'` в загрузчике.

При первом варианте настроек загрузчика получаем мелкие фрагменты.

In [ ]:
from langchain_community.document_loaders import UnstructuredExcelLoader

loader = UnstructuredExcelLoader('din-zatrat-rs.xlsx')
splitted_xlsx = loader.load_and_split(text_splitter)

Посмотрим, сколько всего получилось.

In [ ]:
len(splitted_xlsx)

И отдельно посмотрим один из них.

In [ ]:
splitted_xlsx[5]

Теперь используем настройки по умолчанию.

In [ ]:
splitted_xlsx1 = loader.load_and_split()

Получим другой список с более крупными фрагментами.

In [ ]:
len(splitted_xlsx1)

Посмотрим отдельный фрагмент.

In [ ]:
splitted_xlsx1[3]

Теперь рассмотрим, как будут нарезаны фрагменты, если установить `mode='elements'`. В [документации](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.excel.UnstructuredExcelLoader.html#langchain_community.document_loaders.excel.UnstructuredExcelLoader.load_and_split) сказано, что в этом случае каждый лист становится отдельным элементом. Но после этого происходит еще нарезка внутри одного листа.

In [ ]:
loader = UnstructuredExcelLoader('din-zatrat-rs.xlsx', mode='elements')
splitted_xlsx2 = loader.load()

Список получился большим.

In [ ]:
len(splitted_xlsx2)

Соответственно, каждый фрагмент будет маленьким.

In [ ]:
splitted_xlsx2[5]

### Создание векторных представлений и векторного хранилища
Используем в итоге загрузчик с настройками по умолчанию.

Создадим очередное векторное хранилище.

In [ ]:
vector_store_xlsx = FAISS.from_documents(splitted_xlsx1, embedding=embedding)

Снова зададим вопросы.

#### Вопрос 4
Какой процент расходов на социальную защиту работников в добыче полезных ископаемых был зафиксирован в 2021 году?

#### Правильный ответ
В 2021 году расходы на социальную защиту работников в добыче полезных ископаемых составляли 22,6%.

In [ ]:
vector_store_xlsx.similarity_search(q4)

#### Вопрос 5
Какой процент от общих затрат на рабочую силу составляла заработная плата в 2005 году по всем обследованным видам экономической деятельности?

#### Правильный ответ
В 2005 году заработная плата составляла 76,6% от общих затрат на рабочую силу по всем обследованным видам экономической деятельности.

In [ ]:
vector_store_xlsx.similarity_search(q5)

#### Вопрос 6
В каком году был зафиксирован наибольший процент заработной платы в обрабатывающей промышленности и чему он равен?

#### Правильный ответ
Наибольший процент заработной платы в обрабатывающей промышленности был зафиксирован в 2007 году и составил 79,5%.

In [ ]:
vector_store_xlsx.similarity_search(q6)